In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from  sklearn.model_selection  import  train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

dataset=pd.read_csv('Pima Indians Diabetes Dataset - diabetes.csv')

# Glucose
dataset['Glucose'].replace(0,np.NaN,inplace=True)
mean_Glucose=dataset['Glucose'].mean()
dataset['Glucose'].fillna(mean_Glucose, inplace = True)

# BloodPressure
dataset['BloodPressure'].replace(0,np.NaN,inplace=True)
mean_BloodPressure=dataset['BloodPressure'].mean()
dataset['BloodPressure'].fillna(mean_BloodPressure, inplace = True)

# SkinThickness
dataset['SkinThickness'].replace(0,np.NaN,inplace=True)
mean_SkinThickness=dataset['SkinThickness'].mean()
dataset['SkinThickness'].fillna(mean_SkinThickness, inplace = True)

# Insulin
dataset['Insulin'].replace(0,np.NaN,inplace=True)
mean_Insulin=dataset['Insulin'].mean()
dataset['Insulin'].fillna(mean_Insulin, inplace = True)

# BMI
dataset['BMI'].replace(0,np.NaN,inplace=True)
mean_BMI=dataset['BMI'].mean()
dataset['BMI'].fillna(mean_BMI, inplace = True)

StSc=StandardScaler()
feature=dataset.drop(["Outcome"],axis = 1)
feature=  pd.DataFrame(StSc.fit_transform(feature),columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age'])
outcome=dataset['Outcome']
train_X, test_X, train_Y, test_Y = train_test_split(feature, outcome, test_size=0.1,random_state=0)
train_X,validate_X,train_Y,validate_Y=train_test_split(train_X, train_Y, test_size=0.2,random_state=0)


max_f1=0;
depth_maxf1=0;
criteri_maxf1="entropy";

#Choosing the best model using cross validation by maximizing f1 score
for criteri in ["entropy","gini"]:
    for depth in range(1,20):
        clf = RandomForestClassifier(max_depth=depth,criterion=criteri,random_state=0)
        clf.fit(train_X.values, train_Y.values)
        tn, fp, fn, tp=(metrics.confusion_matrix(validate_Y,clf.predict(validate_X.to_numpy()))).ravel();
        accuracy=(tp+tn)/(tp+tn+fp+fn);
        recall=tp/(tp+fn);
        precisison=tp/(tp+fp)
        f1=2*(recall*precisison)/(precisison+recall)
        if(f1>max_f1):
            max_f1=f1;
            depth_maxf1=depth;
            criteri_maxf1=criteri

        
print("For best result the depth should be: "+str(depth_maxf1)+", Criteria should be "+str(criteri_maxf1))


#Testing the best model with test data
clf = RandomForestClassifier(max_depth=depth_maxf1,criterion=criteri_maxf1,random_state=0)
clf.fit(train_X.values, train_Y.values)
tn, fp, fn, tp=(metrics.confusion_matrix(test_Y,clf.predict(test_X.to_numpy()))).ravel();
print("TN :"+str(tn))
print("FP :"+str(fp))
print("FN :"+str(fn))
print("TP :"+str(tp))
accuracy=(tp+tn)/(tp+tn+fp+fn);
recall=tp/(tp+fn);
precisison=tp/(tp+fp)
f1=2*(recall*precisison)/(precisison+recall)
if(f1>max_f1):
    max_f1=f1;
    depth_maxf1=depth;
    criteri_maxf1=criteri
print("Recall = "+str(recall));
print("Precisison = "+str(precisison));
print("Accuracy = "+str(accuracy));
print("F1 score = "+str(f1))

#Pickle dump for website
pickle.dump(clf, open("diabetes.pkl",'wb'))


For best result the depth should be: 9, Criteria should be entropy
TN :43
FP :8
FN :7
TP :19
Recall = 0.7307692307692307
Precisison = 0.7037037037037037
Accuracy = 0.8051948051948052
F1 score = 0.7169811320754716
